In [1]:
# LARGE, IMBALANCED dataset
#file_location = "wasbs:///mie1628-2019-06-18t21-07-05-930z@mie1628.blob.core.windows.net/MIE1628/data_reduced_reweighted_final_3.csv/part-00000-54f91216-357c-4161-9d25-ee9577740a9c-c000.csv"
# SMALL, BALANCED dataset
file_location = "wasbs:///mie1628-2019-06-18t21-07-05-930z@mie1628.blob.core.windows.net/MIE1628/data_SMALL_balanced.csv/part-00000-0b2544e2-78ed-44d9-808d-1d2e4ec6a8df-c000.csv"

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
25,application_1560963602550_0019,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.
/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/usr/bin/anaconda/envs/py35/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc_learn1').getOrCreate()

In [3]:
# Import the data into a Spark DataFrame with the schema 
data = spark.read.format("csv").option("header","true").option("inferSchema","true")\
.load(file_location)

# Import libraries
from pyspark.sql.functions import isnan, when, count, col

#data = spark.read.csv(file_location, header=True, inferSchema=True)
#data = (data.drop("Census_FirmwareVersionIdentifier")
#    .withColumn("AVProductsEnabled", when(data["AVProductsEnabled"]=="unknown", -1).otherwise(data["AVProductsEnabled"]))
#    .withColumn("classWeightCol", when(data["classWeightCol"]>.5, 0.091).otherwise(0.908)))

from pyspark.sql.types import DoubleType, StringType, IntegerType

data_1 = (data.withColumn("AVProductsEnabled", data["AVProductsEnabled"].cast(IntegerType()))
          .withColumn("AvSigVersion_new", data["AvSigVersion_new"].cast(StringType()))
          .withColumn("Census_OSBuildNumber", data["Census_OSBuildNumber"].cast(StringType()))
          .withColumn("Census_OSBuildRevision", data["Census_OSBuildRevision"].cast(StringType()))
          .withColumn("Census_OSUILocaleIdentifier", data["Census_OSUILocaleIdentifier"].cast(StringType()))
          .withColumn("Census_OSVersion_new", data["Census_OSVersion_new"].cast(StringType()))
          .withColumn("CountryIdentifier", data["CountryIdentifier"].cast(StringType()))
          .withColumn("LocaleEnglishNameIdentifier", data["LocaleEnglishNameIdentifier"].cast(StringType()))
          .withColumn("OsBuild", data["OsBuild"].cast(StringType()))
          .withColumn("OsSuite", data["OsSuite"].cast(StringType())))
data_1 = data_1.withColumnRenamed("HasDetections","label")
#sampling_seed=1111
#data = data_3.sampleBy("HasDetections", fractions={0: .025, 1: .25}, seed=sampling_seed)
#data_reduced.groupBy("HasDetections").count().show()

In [4]:
#data_reduced.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("wasbs:///mie1628-2019-06-18t21-07-05-930z@mie1628.blob.core.windows.net/MIE1628/data_SMALL_balanced.csv")

In [5]:
#file_location = "wasbs:///mie1628-2019-06-18t21-07-05-930z@mie1628.blob.core.windows.net/MIE1628/data_reduced_reweighted_final_3.csv/part-00000-54f91216-357c-4161-9d25-ee9577740a9c-c000.csv"

#file_location = "wasbs:///mie1628-2019-06-18t21-07-05-930z@mie1628.blob.core.windows.net/MIE1628/data_SMALL_balanced.csv/part-00000-0b2544e2-78ed-44d9-808d-1d2e4ec6a8df-c000.csv"

#data = spark.read.csv(file_location, header=True, inferSchema=True)

In [6]:
data_1.columns

['MachineIdentifier', 'EngineVersion', 'AppVersion', 'RtpStateBitfield', 'IsSxsPassiveMode', 'AVProductStatesIdentifier', 'AVProductsInstalled', 'AVProductsEnabled', 'HasTpm', 'CountryIdentifier', 'LocaleEnglishNameIdentifier', 'Platform', 'Processor', 'OsBuild', 'OsSuite', 'OsPlatformSubRelease', 'SkuEdition', 'IsProtected', 'Firewall', 'Census_MDC2FormFactor', 'Census_DeviceFamily', 'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier', 'Census_PrimaryDiskTotalCapacity', 'Census_PrimaryDiskTypeName', 'Census_SystemVolumeTotalCapacity', 'Census_HasOpticalDiskDrive', 'Census_TotalPhysicalRAM', 'Census_ChassisTypeName', 'Census_InternalPrimaryDiagonalDisplaySizeInInches', 'Census_InternalPrimaryDisplayResolutionHorizontal', 'Census_InternalPrimaryDisplayResolutionVertical', 'Census_PowerPlatformRoleName', 'Census_OSArchitecture', 'Census_OSBranch', 'Census_OSBuildNumber', 'Census_OSBuildRevision', 'Census_OSEdition', 'Census_OSSkuName', 'Census_OSInstallTypeName', 'Censu

In [7]:
### Number of nulls or NAN in each column
#numOfnulls = data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns])
### View the number of nulls in each column in a Pandas DataFrame format
#numOfnulls_pandas = numOfnulls.toPandas()
#numOfnulls_pandas.T

In [8]:
# This function drops columns which have more than 10% of their entries as null values
import pyspark.sql.functions as F

def drop_null_columns(df):
    """
    This function drops columns which have more than 10% of their entries as null values.
    :param df: A PySpark DataFrame
    """
    null_counts = df.select([F.count(F.when(isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0].asDict()
    to_drop = [k for k, v in null_counts.items() if v > 0.1*(df.count())]
    df = df.drop(*to_drop)
    return df

In [9]:
# Drop the colums with more than 10% of the records containing and create a new DataFrame "data_1"
#data_1 = drop_null_columns(data)

In [10]:
# Check the number of columns remaining in DF "data_1"
#len(data_1.columns)

In [11]:
##### Check the number of unique values in each column of the dataframe "data_1" and view in a Pandas DataFrame format
#numUniqueVals = data_1.agg(*(F.countDistinct(col(c)).alias(c) for c in data_1.columns))
#numUniqueVals_pandas = numUniqueVals.toPandas()
# numUniqueVals_pandas.T

In [12]:
#### Create a list of columns which have less than 20 unique values
#numUniqueVals_pandas.T
# Cols_with_less_than_20Unique_vals = [col for col, val in numUniqueVals_pandas.iteritems() if numUniqueVals_pandas[col][0]<20 ]
# Cols_with_less_than_20Unique_vals

In [13]:
# Create a new DF "data_2" which contains the columns with less than 20 unique values only
#data_2 = data_1.select(Cols_with_less_than_20Unique_vals)

In [14]:
# Check the number of columns in DF "data_2"
#len(data_2.columns)

In [15]:
# Sanity check: Check that the number of samples (rows) has not changed from the original dataset 

In [16]:
# Drop the rows (records) with NA entries in any of the X columns of "data_2" and save into a new DF "data_3"
data_1 = data_1.dropna(how='any')

In [17]:
# Check the number of rows remaining in DF "data_1"
data_1.select("label").count()

224119

In [18]:
# Create a sample of the data to look at in a Pandas dataframe (the sample contains 5% of the full data)
# data_sample = data_3.sample(False, 0.005)

In [19]:
# Drop the column containing the labels from the features dataframe before vectorizing
data_1b = data_1.drop("label").drop("MachineIdentifier")

In [20]:
# List the data types of the columns in DF "data_3b"
data_1.dtypes

[('MachineIdentifier', 'string'), ('EngineVersion', 'string'), ('AppVersion', 'string'), ('RtpStateBitfield', 'string'), ('IsSxsPassiveMode', 'int'), ('AVProductStatesIdentifier', 'string'), ('AVProductsInstalled', 'int'), ('AVProductsEnabled', 'int'), ('HasTpm', 'int'), ('CountryIdentifier', 'string'), ('LocaleEnglishNameIdentifier', 'string'), ('Platform', 'string'), ('Processor', 'string'), ('OsBuild', 'string'), ('OsSuite', 'string'), ('OsPlatformSubRelease', 'string'), ('SkuEdition', 'string'), ('IsProtected', 'string'), ('Firewall', 'string'), ('Census_MDC2FormFactor', 'string'), ('Census_DeviceFamily', 'string'), ('Census_ProcessorCoreCount', 'double'), ('Census_ProcessorManufacturerIdentifier', 'string'), ('Census_PrimaryDiskTotalCapacity', 'int'), ('Census_PrimaryDiskTypeName', 'string'), ('Census_SystemVolumeTotalCapacity', 'double'), ('Census_HasOpticalDiskDrive', 'int'), ('Census_TotalPhysicalRAM', 'double'), ('Census_ChassisTypeName', 'string'), ('Census_InternalPrimaryDia

In [21]:
stringCols = []
for col in data_1.dtypes:
    if col[1] == 'string':
        stringCols.append(col[0])

In [22]:
stringCols.pop(0)
stringCols

['EngineVersion', 'AppVersion', 'RtpStateBitfield', 'AVProductStatesIdentifier', 'CountryIdentifier', 'LocaleEnglishNameIdentifier', 'Platform', 'Processor', 'OsBuild', 'OsSuite', 'OsPlatformSubRelease', 'SkuEdition', 'IsProtected', 'Firewall', 'Census_MDC2FormFactor', 'Census_DeviceFamily', 'Census_ProcessorManufacturerIdentifier', 'Census_PrimaryDiskTypeName', 'Census_ChassisTypeName', 'Census_PowerPlatformRoleName', 'Census_OSArchitecture', 'Census_OSBranch', 'Census_OSBuildNumber', 'Census_OSBuildRevision', 'Census_OSEdition', 'Census_OSSkuName', 'Census_OSInstallTypeName', 'Census_OSInstallLanguageIdentifier', 'Census_OSUILocaleIdentifier', 'Census_OSWUAutoUpdateOptionsName', 'Census_GenuineStateName', 'Census_ActivationChannel', 'Census_FlightRing', 'Census_FirmwareManufacturerIdentifier', 'Census_IsVirtualDevice', 'Census_IsAlwaysOnAlwaysConnectedCapable', 'Wdft_IsGamer', 'Wdft_RegionIdentifier', 'AvSigVersion_new', 'OSVer_new', 'Census_OSVersion_new', 'OsBuildLab_new']

In [23]:
import numpy as np
numericCols = np.setdiff1d(data_1.columns, stringCols).tolist()
numericCols.remove("MachineIdentifier")
numericCols.remove("classWeightCol")
numericCols.remove("label")

In [24]:
data_1c = data_1.select(stringCols).drop("MachineIdentifier")

#### Encode categorical features using StringIndexer

In [25]:
# Import libraries
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

In [26]:
# Use StringIndexer to create numerical categorical columns for the columns of dataFrame "data_3b"
#indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(data_1c) \
#            for column in data_1c.columns]

from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler

# The index of string values multiple columns
indexers = [
    StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c))
    for c in stringCols
]

# The encode of indexed vlaues multiple columns
encoders = [OneHotEncoder(dropLast=False,inputCol=indexer.getOutputCol(),
            outputCol="{0}_encoded".format(indexer.getOutputCol())) 
    for indexer in indexers
]

# Vectorizing encoded values
assembler = VectorAssembler(inputCols=([encoder.getOutputCol() for encoder in encoders] + numericCols),outputCol="features")

pipeline = Pipeline(stages=indexers + encoders+[assembler])
model=pipeline.fit(data_1)
transformed = model.transform(data_1)

In [27]:
transformed.select("features").show(1)

+--------------------+
|            features|
+--------------------+
|(5351,[1,49,128,1...|
+--------------------+
only showing top 1 row

### One-hot encoding maps a categorical feature, represented as a label index, to a binary vector with at most a single one-value indicating the presence of a specific feature value from among the set of all feature values. This encoding allows algorithms which expect continuous features, such as Logistic Regression, to use categorical features. For string type input data, it is common to encode categorical features using StringIndexer first (done above).

In [28]:
#data_5b.columns

In [29]:
# Create a dataFrame called "final_df_1" composed of two columns:
# the first column is called "features" and it includes the feature vector for each record (which is a vector produced by one hot encoding then vector assembler)
# the second column is the "Has detection" column which is basically the label for each record 
dataset = transformed.select('features','label','classWeightCol')
dataset.columns

['features', 'label', 'classWeightCol']

In [30]:
#dataset = final_df_1.crossJoin(data_3.select("HasDetections")).crossJoin(data_3.select("classWeightCol"))


In [31]:
#dataset.count()

In [32]:
# Create a sample of the data to look at in a Pandas dataframe (the sample contains 5% of the full data)
# dataset_sample = dataset.sample(False, 0.02)

In [33]:
# dataset_sample.count()

In [34]:
# dataset_sample.show(5)

In [35]:
#final_df_3.write.save('/user/mie_aabdelmageed/final_df_3', format='parquet', mode='append')

In [36]:
#Good = dataset.filter(dataset.HasDetections == 0)

In [37]:
#Good.show(5)

In [38]:
#Good.count()

In [39]:
#noGood = dataset.filter(dataset.HasDetections == 1)

In [40]:
#noGood.count()

## Stratified Split

In [41]:
sampling_seed=1111
trainingData = dataset.sampleBy("label", fractions={0: .9, 1: .9}, seed=sampling_seed)

# Subtracting 'train' from original 'data' to get test set 
testData = dataset.subtract(trainingData)

In [42]:
trainingData.show(5)

+--------------------+-----+--------------+
|            features|label|classWeightCol|
+--------------------+-----+--------------+
|(5351,[1,65,128,1...|    1|         0.908|
|(5351,[1,54,128,1...|    1|         0.908|
|(5351,[9,48,128,3...|    0|         0.091|
|(5351,[2,57,128,1...|    0|         0.091|
|(5351,[0,54,128,1...|    1|         0.908|
+--------------------+-----+--------------+
only showing top 5 rows

In [43]:
from pyspark.ml.classification import LogisticRegression

In [44]:
### Randomly split data into training and test sets. set seed for reproducibility
#(trainingData, testData) = dataset_sample.randomSplit([0.8, 0.2], seed=100)
#print(trainingData.count())
#print(testData.count())

In [45]:
# Create initial LogisticRegression model
lr = (LogisticRegression(labelCol="label", featuresCol="features",
                         maxIter=100, regParam=0.1, elasticNetParam=0.5))
                        
#lr = LogisticRegression(labelCol="HasDetections", featuresCol="features", weightCol="classWeightCol", maxIter=10)

# Train model with Training Data
lrModel = lr.fit(trainingData)

In [46]:
# Make predictions on test data using the transform() method.
# LogisticRegression.transform() will only use the 'features' column.
predictions = lrModel.transform(testData)

In [47]:
predictions.show()

+--------------------+-----+--------------+--------------------+--------------------+----------+
|            features|label|classWeightCol|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------+--------------------+--------------------+----------+
|(5351,[2,56,128,1...|    1|         0.908|[-0.0642615610951...|[0.48394013601111...|       1.0|
|(5351,[1,48,128,1...|    1|         0.908|[-0.0642615610951...|[0.48394013601111...|       1.0|
|(5351,[0,48,129,2...|    0|         0.091|[0.14625370819804...|[0.53649839140106...|       0.0|
|(5351,[0,48,128,1...|    0|         0.091|[-0.0642615610951...|[0.48394013601111...|       1.0|
|(5351,[1,50,128,1...|    1|         0.908|[-0.0642615610951...|[0.48394013601111...|       1.0|
|(5351,[6,79,128,1...|    1|         0.908|[-0.0642615610951...|[0.48394013601111...|       1.0|
|(5351,[0,57,128,1...|    0|         0.091|[0.12929520077493...|[0.53227884492719...|       0.0|
|(5351,[24,68,128,...|    1|  

In [48]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
evaluator.evaluate(predictions)

0.5724250595964253

In [49]:
####Cross Validation

### Feature Selection using ChiSqSelector
ChiSqSelector stands for Chi-Squared feature selection. It operates on labeled data with categorical features. ChiSqSelector uses the Chi-Squared test of independence to decide which features to choose

In [50]:
from pyspark.ml.feature import ChiSqSelector
# from pyspark.ml.linalg import Vectors

In [51]:
# Create a chiSq selector instance to select the top 50 most important features in the dataset "final_df_3"
selector_1 = ChiSqSelector(numTopFeatures=50, featuresCol="features", \
                         outputCol="selectedFeatures", labelCol="label")

In [52]:
final_df_3_chi = selector_1.fit(dataset).transform(dataset)

'Field "HasDetections" does not exist.\nAvailable fields: features, label, classWeightCol'
Traceback (most recent call last):
  File "/usr/hdp/current/spark2-client/python/pyspark/ml/base.py", line 132, in fit
    return self._fit(dataset)
  File "/usr/hdp/current/spark2-client/python/pyspark/ml/wrapper.py", line 288, in _fit
    java_model = self._fit_java(dataset)
  File "/usr/hdp/current/spark2-client/python/pyspark/ml/wrapper.py", line 285, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/usr/hdp/current/spark2-client/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/hdp/current/spark2-client/python/pyspark/sql/utils.py", line 79, in deco
    raise IllegalArgumentException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.IllegalArgumentException: 'Field "HasDetections" does not exist.\nAvailable fields: features, label, classWeightCol'



In [53]:
final_df_3_chi.show(5)

name 'final_df_3_chi' is not defined
Traceback (most recent call last):
NameError: name 'final_df_3_chi' is not defined



In [54]:
# final_df_3_chi.write.save('/user/mie_aabdelmageed/final_df_3_chi', format='parquet', mode='append')

## Gradient Boosted Trees

In [55]:
from pyspark.ml.classification import GBTClassifier


gbt = GBTClassifier(labelCol="HasDetections", featuresCol="features", maxIter=10)
GBT_Model = gbt.fit(trainingData)
gbt_predictions = GBT_Model.transform(testData)
evaluator = BinaryClassificationEvaluator()


print("Test_SET (Area Under ROC): " + str(evaluator.evaluate(gbt_predictions, {evaluator.metricName: "areaUnderROC"})))


'Field "HasDetections" does not exist.\nAvailable fields: features, label, classWeightCol'
Traceback (most recent call last):
  File "/usr/hdp/current/spark2-client/python/pyspark/ml/base.py", line 132, in fit
    return self._fit(dataset)
  File "/usr/hdp/current/spark2-client/python/pyspark/ml/wrapper.py", line 288, in _fit
    java_model = self._fit_java(dataset)
  File "/usr/hdp/current/spark2-client/python/pyspark/ml/wrapper.py", line 285, in _fit_java
    return self._java_obj.fit(dataset._jdf)
  File "/usr/hdp/current/spark2-client/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py", line 1257, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/usr/hdp/current/spark2-client/python/pyspark/sql/utils.py", line 79, in deco
    raise IllegalArgumentException(s.split(': ', 1)[1], stackTrace)
pyspark.sql.utils.IllegalArgumentException: 'Field "HasDetections" does not exist.\nAvailable fields: features, label, classWeightCol'

